In [ ]:
import pandas as pd
import numpy as np
import logging
import shared
import os

os.environ['RACE_TYPE'] = "ju"
race_type = shared.race_type()
#year = shared.forecast_year()
import time
startTime = time.time()

In [ ]:
cleaned = pd.read_csv(f'Jukola-terrain/ideal-paces-{race_type}.tsv', delimiter='\t')

In [ ]:
cleaned.sort_values(by=["year", "leg"], ascending=[False, True]).head(10)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#sns.set(rc={"figure.figsize":(16, 9)}) 
plot = sns.scatterplot(x="leg_distance", y="ideal_pace", hue="year", palette="bright", data=cleaned)



In [ ]:

ax = sns.boxplot(x="year", y="ideal_pace", data=cleaned)

In [ ]:

ax = sns.boxplot(x="leg", y="ideal_pace", data=cleaned)

In [ ]:
ax = sns.boxplot(x="year", y="marking", data=cleaned)

In [ ]:
ax = sns.boxplot(x="year", y="vertical_per_km", data=cleaned[cleaned["year"] >= 2015])

In [ ]:
plot = sns.scatterplot(x="vertical_per_km", y="ideal_pace", hue="year", palette="bright", data=cleaned[cleaned["year"] >= 2015])


In [ ]:
cleaned

In [ ]:
leg_means = cleaned[["leg", "ideal_pace"]].groupby(["leg"]).agg("mean")
leg_means

In [ ]:
ax = sns.scatterplot(x="year", y="terrain_coefficient", hue="leg", data=cleaned)

In [ ]:
ax = sns.scatterplot(x="leg", y="terrain_coefficient", hue="year", data=cleaned)

In [ ]:
# TODO get all runs post race
runs = pd.read_csv(f'data/runs_{shared.race_id_str()}.tsv', delimiter='\t')
runs = runs.query("num_runs > 1")

In [ ]:
runner_means = runs[["name", "pace"]].groupby(["name"]).agg("mean")
runner_means


In [ ]:
runs["pace_mean"] = runner_means["pace"][runs["name"]].values
runs["personal_coefficient"] = runs["pace"] / runs["pace_mean"]

runs.head()

In [ ]:
runs = pd.merge(runs, cleaned[["year", "leg", "terrain_coefficient", "marking", "vertical", "vertical_per_km"]], how="left", on=["year", "leg"])
runs["personal_terain_diff"] = runs["personal_coefficient"] - runs["terrain_coefficient"] 
runs.round(4)

In [ ]:
ax = sns.lmplot(x="terrain_coefficient", y="personal_coefficient", data=runs, height=8, aspect=1.7, ci=50,scatter_kws={'alpha':0.07}, x_jitter=0.0004)
ax.set(ylim=(0.6, 1.6))

In [ ]:
ax = sns.residplot(x="terrain_coefficient", y="personal_coefficient", lowess=True, data=runs,scatter_kws={'alpha':0.07})

In [ ]:
ax = sns.lmplot(x="terrain_coefficient", y="personal_coefficient", hue="leg",data=runs, height=8, aspect=1.7, ci=50,scatter_kws={'alpha':0.07}, x_jitter=0.0003)
ax.set(ylim=(0.6, 1.6))

In [ ]:
ax = sns.lmplot(x="terrain_coefficient", y="personal_coefficient", hue="year",data=runs, height=8, aspect=1.7, ci=50,scatter_kws={'alpha':0.07}, x_jitter=0.0003)
ax.set(ylim=(0.6, 1.6))

In [ ]:
ax = sns.lmplot(x="marking", y="personal_coefficient", data=runs, height=8, aspect=1.7, ci=50,scatter_kws={'alpha':0.07}, x_jitter=3)
ax.set(ylim=(0.6, 1.6))

In [ ]:
ax = sns.lmplot(x="vertical", y="personal_coefficient", data=runs, height=8, aspect=1.7, ci=50,scatter_kws={'alpha':0.07}, x_jitter=1)
ax.set(ylim=(0.6, 1.6))

In [ ]:
ax = sns.lmplot(x="vertical_per_km", y="personal_coefficient", data=runs, height=8, aspect=1.7, ci=50,scatter_kws={'alpha':0.07}, x_jitter=0.1)
ax.set(ylim=(0.6, 1.6))

In [ ]:
ax = sns.boxplot(x="year", y="personal_terain_diff", data=runs.query("personal_terain_diff < 0.4 & personal_terain_diff > -0.3 & num_runs < 10"))


In [ ]:
ax = sns.lmplot(x="terrain_coefficient", y="personal_coefficient",row="leg", data=runs)

In [ ]:
ax = sns.lmplot(x="terrain_coefficient", y="personal_coefficient",row="year", data=runs)

In [ ]:
#
runs["log_team_id"]= np.log(runs["team_id"])
ax = sns.lmplot(x="log_team_id", y="personal_coefficient", data=runs)

In [ ]:
runner_name = "nelly carlsson"
if race_type == "ju":
    runner_name = "topi anjala"
    runner_name = "juha meronen"
    
runner = runs[runs["name"] == runner_name]
ax = sns.lmplot(x="terrain_coefficient", y="personal_coefficient",data=runner)

In [ ]:
runner.round(3)

In [ ]:
runner["terrain_coefficient"].values

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'p_%s' % n
    return percentile_
